In [5]:
import os
import dlib
from skimage import io
import csv
import numpy as np

# Traning data
path_images_from_camera = "../data/faces/"

# Dlib face forward face detector
detector = dlib.get_frontal_face_detector()

# Dlib face landmark feature recognization 
predictor = dlib.shape_predictor('../data/shape_predictor_68_face_landmarks.dat')

# Dlib Resnet Face Recognization Model, extrect 128-d feature vectors
face_reco_model = dlib.face_recognition_model_v1("../data/dlib_face_recognition_resnet_model_v1.dat")

In [6]:
# Return 128D features for single image
# Input:    path_img           <class 'str'>
# Output:   face_descriptor    <class 'dlib.vector'>
def return_128d_features(path_img):
    img_rd = io.imread(path_img)
    faces = detector(img_rd, 1)

    print("%-40s %-20s" % (" >> Image with faces detected:", path_img), '\n')

    # Make sure that the captured picture is a human face
    if len(faces) != 0:
        shape = predictor(img_rd, faces[0])
        face_descriptor = face_reco_model.compute_face_descriptor(img_rd, shape)
    else:
        face_descriptor = 0
        print("no face")
    return face_descriptor



In [7]:
# Return the mean value of 128D face descriptor for person X
# Input:    path_faces_personX       <class 'str'>      Folder path of each person
# Output:   features_mean_personX    <class 'numpy.ndarray'>

def return_features_mean_personX(path_faces_personX):
    features_list_personX = []
    photos_list = os.listdir(path_faces_personX)  #face_1-10
    for i in photos_list:
        if i.split('/')[-1][0]=='.':
            photos_list.remove(i)
    if photos_list: 
        for i in range(len(photos_list)): #0-9
            # return_128d_features()
            print("%-40s %-20s" % (" >> 正在读的人脸图像 / Reading image:", path_faces_personX + "/" + photos_list[i]))
            features_128d = return_128d_features(path_faces_personX + "/" + photos_list[i])
            # Skip when encountering a picture that does not detect a face
            if features_128d == 0:
                i += 1
            else:
                features_list_personX.append(features_128d)
    else:
        print(" >>  Warning: No images in " + path_faces_personX + '/', '\n')

    # Compute mean of 128-d features
    # 10 images of personX x 128D -> 1 x 128D
    if features_list_personX:
        features_mean_personX = np.array(features_list_personX).mean(axis=0)
    else:
        features_mean_personX = np.zeros(128, dtype=int, order='C')
    return features_mean_personX


In [ ]:
import json
# Get the 128D feature vectors for each person and put them into a csv file
labels=[]  # Corresponding to the name of each person in the CSV file
labelFile=open('../data/label.txt','w')
with open("../data/features_all.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for person in os.listdir(path_images_from_camera):
        print(path_images_from_camera + str(person))
        if (path_images_from_camera + str(person)).split('/')[-1][0]!='.':
            features_mean_personX = return_features_mean_personX(path_images_from_camera + str(person))
            labels.append(str(person))
            writer.writerow(features_mean_personX)
            print(" >> The mean of features:", list(features_mean_personX), '\n')
    print("Save all the features of faces registered into: data/features_all.csv")
json.dump(labels,labelFile)
labelFile.close()

In [ ]:
# Real-time face detection and recognition
import json
import dlib
import numpy as np
import cv2
import pandas as pd
import os
import time
from PIL import Image, ImageDraw, ImageFont


class Face_Recognizer:
    def __init__(self):
        self.feature_known_list = []                # all the entered facial features
        self.name_known_list = []                   # all the entered names

        self.current_frame_face_cnt = 0             # the number of faces captured in the current camera
        self.current_frame_feature_list = []        # face features captured in the current camera
        self.current_frame_name_position_list = []  # the name coordinates of all faces captured in the current camera
        self.current_frame_name_list = []           # the names of all faces captured in the current camera
        # Update FPS
        self.fps = 0
        self.frame_start_time = 0

    # Get face features from "features_all.csv" 
    def get_face_database(self):
        if os.path.exists("/../data/features_all.csv"):
            path_features_known_csv = "../data/features_all.csv"
            csv_rd = pd.read_csv(path_features_known_csv, header=None)
            for i in range(csv_rd.shape[0]):
                features_someone_arr = []
                for j in range(0, 128):
                    if csv_rd.iloc[i][j] == '':
                        features_someone_arr.append('0')
                    else:
                        features_someone_arr.append(csv_rd.iloc[i][j])
                self.feature_known_list.append(features_someone_arr)
                self.name_known_list.append(labels[i])
            print("Faces in Database：", len(self.feature_known_list))
            return 1
        else:
            print('##### Warning #####', '\n')
            print("'features_all.csv' not found!")
            print(
                "Please run 'get_faces_from_camera.py' and 'features_extraction_to_csv.py' before 'face_reco_from_camera.py'",
                '\n')
            print('##### End Warning #####')
            return 0

    # Calculate the Euclidean distance between two 128D vectors 
    @staticmethod
    def return_euclidean_distance(feature_1, feature_2):
        feature_1 = np.array(feature_1)
        feature_2 = np.array(feature_2)
        dist = np.sqrt(np.sum(np.square(feature_1 - feature_2)))
        return dist

    # update FPS 
    def update_fps(self):
        now = time.time()
        self.frame_time = now - self.frame_start_time
        self.fps = 1.0 / self.frame_time
        self.frame_start_time = now

    def draw_note(self, img_rd):
        font = cv2.FONT_ITALIC

        cv2.putText(img_rd, "Face Recognizer", (20, 40), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "FPS:   " + str(self.fps.__round__(2)), (20, 100), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "Faces: " + str(self.current_frame_face_cnt), (20, 140), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "Q: Quit", (20, 450), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    def draw_name(self, img_rd):
        # Draw names under rectangle
        #font = ImageFont.truetype("simsun.ttc", 30)
        img = Image.fromarray(cv2.cvtColor(img_rd, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img)
        for i in range(self.current_frame_face_cnt):
            # cv2.putText(img_rd, self.current_frame_name_list[i], self.current_frame_name_position_list[i], font, 0.8, (0, 255, 255), 1, cv2.LINE_AA)
            #textsize = 20
            #ft = ImageFont.truetype(self.current_frame_name_list[i], textsize)
           # draw.text(xy=self.current_frame_name_position_list[i], text=self.current_frame_name_list[i])
            draw.text(xy=self.current_frame_name_position_list[i], text=self.current_frame_name_list[i])
            img_with_name = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        return img_with_name

    # Process the acquired video stream for face recognition
    def process(self, stream):
        # 1. Read the CSV file of all facial features
        if self.get_face_database():
            while stream.isOpened():
                print(">>> Frame start")
                flag, img_rd = stream.read()
                faces = detector(img_rd, 0)
                kk = cv2.waitKey(1)
                #  Press 'q' to quit
                if kk == ord('q'):
                    break
                else:
                    self.draw_note(img_rd)
                    self.current_frame_feature_list = []
                    self.current_frame_face_cnt = 0
                    self.current_frame_name_position_list = []
                    self.current_frame_name_list = []

                    # 2. Recognization of faces
                    if len(faces) != 0:
                        # 3. Gets the features of all faces of the currently captured image
                        for i in range(len(faces)):
                            shape = predictor(img_rd, faces[i])
                            self.current_frame_feature_list.append(face_reco_model.compute_face_descriptor(img_rd, shape))
                        # 4. Iterate through all faces in the captured image
                        for k in range(len(faces)):
                            print(">>>>>> For face", k+1, " in camera")
                            # Default everyone is unknown 
                            self.current_frame_name_list.append("unknown")

                            # The coordinates of the name of each captured face
                            self.current_frame_name_position_list.append(tuple(
                                [faces[k].left(), int(faces[k].bottom() + (faces[k].bottom() - faces[k].top()) / 4)]))

                            # 5. For each face, traverse through all stored face features
                            current_frame_e_distance_list = []
                            for i in range(len(self.feature_known_list)):
                                if str(self.feature_known_list[i][0]) != '0.0':
                                    print("   >>> With person", str(i + 1), ", the e distance: ", end='')
                                    e_distance_tmp = self.return_euclidean_distance(self.current_frame_feature_list[k],
                                                                                    self.feature_known_list[i])
                                    print(e_distance_tmp)
                                    current_frame_e_distance_list.append(e_distance_tmp)
                                else:
                                    # empty person_X
                                    current_frame_e_distance_list.append(999999999)
                            # 6. Find the smallest Euclidean distance match
                            similar_person_num = current_frame_e_distance_list.index(min(current_frame_e_distance_list))
                            print("   >>> Minimum e distance with ", self.name_known_list[similar_person_num], ": ", min(current_frame_e_distance_list))

                            if min(current_frame_e_distance_list) < 0.9:
                                self.current_frame_name_list[k] = self.name_known_list[similar_person_num]
                                print("   >>> Face recognition result:  " + str(self.name_known_list[similar_person_num]))
                            else:
                                print("   >>> Face recognition result: Unknown person")

                            # Rectangle
                            for kk, d in enumerate(faces):
                                cv2.rectangle(img_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]),
                                              (0, 255, 255), 2)

                        self.current_frame_face_cnt = len(faces)
                       

                        # 7. Write names
                        img_with_name = self.draw_name(img_rd)

                    else:
                        img_with_name = img_rd

                print(">>>>>> Faces in camera now:", self.current_frame_name_list)

                cv2.imshow("camera", img_with_name)

                # 8. Update stream FPS
                self.update_fps()
                print(">>> Frame ends\n\n")

    # OpenCV call the camera
    def run(self):
        cap = cv2.VideoCapture(0)
        # cap = cv2.VideoCapture("video.mp4")
        cap.set(3, 480)     # 640x480
        self.process(cap)

        cap.release()
        cv2.destroyAllWindows()


def main():
    Face_Recognizer_con = Face_Recognizer()
    Face_Recognizer_con.run()


if __name__ == '__main__':
    label=open('../data/label.txt','r')
    labels=json.load(label)
    main()
    label.close()
